In [1]:
using CSV
using Dates
using DataFrames

In [2]:
using JuMP
using Gurobi

In [3]:
function patient_allocation(beds::Array{Float32,1}, initial_patients::Array{Float32,1}, net_patients::Array{Float32,2})
    N, T = size(net_patients)
    @assert(size(beds, 1) == N)

    model = Model(Gurobi.Optimizer)
    @variable(model, sent[1:N,1:N,1:T] >= 0)
    @variable(model, dummy[1:N,1:T] >= 0)

    @objective(model, Min, sum(dummy))

    # sent <= current_patients
    @constraint(model, [t=1:T],
        sum(sent[:,:,t], dims=2) .<=
            initial_patients
            .+ sum(net_patients[:,1:t], dims=2)
            .- sum(sent[:,:,1:t-1], dims=[2,3])
            .+ sum(sent[:,:,1:t-1], dims=[1,3])
    )

    # can't send to self
    @constraint(model, [i=1:N],
        sent[i,i,:] .== 0
    )

    # dummy
    @constraint(model, [i=1:N,t=1:T],
        dummy[i,t] >= (initial_patients[i] + sum(net_patients[i,1:t]) - sum(sent[i,:,1:t]) + sum(sent[:,i,1:t])) - beds[i]
    )

    optimize!(model)
    return model
end;

In [19]:
# select start and end dates
start_date = Date(2020, 4, 1)
end_date   = Date(2020, 5, 15)

# load the forecast data
forecast_data = CSV.read("../../data/forecasts/ihme_2020_04_12/forecast.csv", copycols=true)

# filter to US states
state_list = CSV.read("../../data/geography/state_names.csv", copycols=true)
filter!(row -> row.location_name in state_list.State, forecast_data)

# add state abbreviations
state_dict = Dict(state.State => state.Abbreviation for state in eachrow(state_list))
forecast_data.state = [state_dict[row.location_name] for row in eachrow(forecast_data)]

# sort
sort!(forecast_data, [:state, :date])

# compute net change
allbed_mean_net = Array{Float64,1}(undef, size(forecast_data,1))
forecast_start = Array{Float32,1}(undef, length(unique(forecast_data.state)))
groups = groupby(forecast_data, :state).groups
for i = 1:maximum(groups)
    mask = groups .== i
    rows = forecast_data[mask,:]
    allbed_mean_net[mask] = rows.allbed_mean - [0; rows.allbed_mean[1:end-1]]
    forecast_start[i] = rows.allbed_mean[rows.date .== start_date-Dates.Day(1)][1]
end
insertcols!(forecast_data, 1, :allbed_mean_net => allbed_mean_net)

# filter by date
filter!(row -> start_date <= row.date <= end_date, forecast_data)

# group by state
forecast_data_loc = groupby(forecast_data, :state, sort=true)

# select forecast column
forecast = hcat([loc.allbed_mean_net[:] for loc in forecast_data_loc]...)'
forecast = Float32.(forecast);

In [145]:
# load the beds data
beds_data = CSV.read("../../data/hospitals/hospital_locations.csv", copycols=true)

# filter
filter!(row -> row.BEDS > 0, beds_data)
filter!(row -> row.STATE in state_list.Abbreviation, beds_data)
filter!(row -> row.TYPE == "GENERAL ACUTE CARE", beds_data)

beds_data.BEDS = beds_data.BEDS * (1/4)

# aggregate by state
beds_data = by(beds_data, :STATE, :BEDS => sum)

# reorder states
sort!(beds_data, :STATE)

# select beds column
beds = beds_data.BEDS_sum;
beds = Float32.(beds);

In [155]:
# load the city distances data
dist_df = CSV.read("../../data/geography/us_city_travel_times.csv", copycols=true, type=String);
dist_data = Matrix(dist_df[:,2:end]);

# convert distances to seconds
function cvt_duration(s::Union{String,Missing})
    if ismissing(s) return 0 end
    parts = parse.(Int, split(s, ":"))
    return 3600*parts[1] + 60*parts[2] + parts[3]
end
city_distances = cvt_duration.(dist_data);

# build list of cities to use
capital_cities = Dict(state.Abbreviation => state.Capitol * ", " * state.Abbreviation for state in eachrow(state_list))
alt_cities = Dict(
    "AK" => "Anchorage, AK",
    "DE" => "Wilmington, DE",
    "KY" => "Louisville, KY",
    "ME" => "Portland, ME",
    "MD" => "Baltimore, MD",
    "MO" => "Kansas City, MO",
    "MT" => "Billings, MT",
    "NH" => "Manchester, NH",
    "SD" => "Sioux Falls, SD",
    "VT" => "Manchester, NH"
);
selected_cities = copy(capital_cities)
for (k,v) in alt_cities
    selected_cities[k] = v
end
selected_city_names = collect(values(selected_cities))
sort!(selected_city_names, by=n -> split(n,", ")[end])

# get indicies of selected cities
matrix_cities = String.(names(dist_df)[2:end])
selected_cities_ind = [findfirst(x -> x == city_name, matrix_cities) for city_name in selected_city_names]

# compute distance matrix
selected_city_distances = city_distances[selected_cities_ind, selected_cities_ind]

# threshold distance matrix to get adjacency matrix
dist_threshold = 3600
adj_matrix = 0 .< selected_city_distances .<= dist_threshold;

thread = 1 warning: only found 1000 / 1001 columns on data row: 1000. Filling remaining columns with `missing`


In [26]:
n_start = 15
n, t = 20, 40
beds_small, forecast_small, forecast_start_small = beds[n_start:n_start+n], forecast[n_start:n_start+n,1:t], forecast_start[n_start:n_start+n];

In [27]:
model = patient_allocation(beds_small, forecast_start_small, forecast_small);

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 19320 rows, 18480 columns and 14820120 nonzeros
Model fingerprint: 0xc66bfe13
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+03]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 1260 rows and 840 columns (presolve time = 5s) ...
Presolve removed 1260 rows and 840 columns
Presolve time: 7.16s
Presolved: 18060 rows, 17640 columns, 14138040 nonzeros


Barrier performed 0 iterations in 12.11 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 1201 iterations and 12.97 seconds
Optimal objective  0.000000000e+00


In [29]:
sent = value.(model[:sent]);
outcomes = DataFrame(
    state=state_list.Abbreviation[n_start:n_start+n],
    total_sent=sum(sent, dims=[2,3])[:],
    total_received=sum(sent, dims=[1,3])[:]
)

,state,total_sent,total_received
,String,Float64,Float64
1,FL,5617.99,7012.9
2,GA,4757.86,2139.79
3,HI,2042.23,2139.79
4,IA,1854.75,2139.79
5,ID,1904.56,2139.79
6,IL,1158.19,2139.79
7,IN,2052.22,2139.79
8,KS,1911.38,2248.13
9,KY,2322.47,2139.79


In [158]:
i = 1
println("Sent/Recieved by ", state_list.State[n_start+i-1], " over time.")
for (x, y) in zip(sum(sent[i,:,:], dims=1), sum(sent[:,i,:], dims=1))
    println(x, " ", y)
end

Sent/Recieved by Florida over time.
0.0 0.0
0.0 0.0
0.0 0.0
0.0 209.52374267578125
0.0 681.3963012695312
0.0 252.12000331878627
0.0 70.265869140625
0.0 108.3282470703125
0.0 0.47609901428222656
596.112147402763 273.09467327594723
0.0 305.7292544603349
0.0 218.288330078125
0.0 840.9069259643555
0.0 545.1215817928314
0.0 211.270263671875
767.3404797792433 0.0
0.0 220.487060546875
0.0 0.0
0.0 200.83642578125
0.0 187.2314453125
442.2668406089149 170.57275390625
73.14941915671008 0.0
0.0 0.0
715.3693885803216 609.3737831115716
185.917236328125 81.96923828125
211.87903738816647 113.43787527879147
188.61807155609202 0.0
0.0 5.795073509216195
148.0644531249994 0.0
0.0 0.0
377.1276855468743 18.129592100779092
294.93718225955945 0.0
341.8447265624999 0.0
276.97558593750045 0.0
998.392054239908 0.0
0.0 0.0
0.0 0.0
0.0 1688.5483144124353
0.0 0.0
0.0 0.0
